<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Use Spark and Python to Predict Customer Churns</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
</table>

This notebook contains steps and code to get data, create a predictive model, and start scoring new data. This notebook introduces commands for getting data and for basic data cleaning and exploration, pipeline creation, model training, model persistance to the Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.5 and Apache Spark 2.3.

You will use the available data set, **CustomerChurn-WS.csv**, which details anonymous customer and churn information. Use the details of this data set to predict customer churn's risk. 

## Learning goals

You will learn how to:

-  Load a CSV file into an Apache Spark DataFrame.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create an Apache Spark machine learning pipeline.
-  Train and evaluate a model.
-  Store a pipeline and model in Watson Machine Learning (WML) repository.
-  Deploy a model for online scoring using the Watson Machine Learning (WML) API.
-  Score sample scoring data using the WML API.

## Contents

This notebook contains the following parts:

1.	[Set up](#setup)
2.	[Load and explore data](#load)
3.	[Create a spark ml model](#model)
4.	[Store the model](#persistence)
5.	[Predict locally](#prediction)
6.	[Deploy and score in a Cloud](#scoring)
7.	[Summary and next steps](#summary)
    
**If you are new to data science check out this link:** https://developer.ibm.com/articles/ba-intro-data-science-1/

<a id="setup"></a>
## 1. Set up

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a lite plan is offered and information about how to create the instance is <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">here</a>)
-  Make sure that you are using a Spark 2.3 kernel
-  Download the **Customer Churn data** (code provided below)

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache Spark DataFrame and perform a basic exploration.

Load the data to the Spark DataFrame by using *wget* to upload the data to gpfs and then use spark *read* method to read data. 

In [1]:
# Install wget if you don't already have it.
!pip install wget

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20180920234211-0001
Target directory /home/spark/shared/user-libs/python3/wget-3.2.dist-info already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3/wget.py already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3/__pycache__ already exists. Specify --upgrade to force replacement.


In [2]:
import wget

link_to_data = 'https://raw.githubusercontent.com/huisim/DataDSX/master/CustomerChurn-WS.csv'
filename = wget.download(link_to_data)

print(filename)

CustomerChurn-WS (1).csv


The csv file **CustomerChurn-WS.csv** is now availble. Load the file to Apache Spark DataFrame using code below.

In [3]:
#spark = SparkSession.builder.getOrCreate()

df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load(filename)

Explore the loaded data by using the following Apache Spark DataFrame methods:
-  print schema
-  remove spaces within column names
-  print top 20 records
-  count all records

In [4]:
df_data.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Est Income: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- LocalBilltype: string (nullable = true)
 |-- LongDistanceBilltype: string (nullable = true)
 |-- Usage: double (nullable = true)
 |-- CHURN: string (nullable = true)



As you can see, the data contains 9 fields. CHURN field is the one you would like to predict (label).

In [5]:
# Remove spaces from columns names, it's an example of data preparation that you may have to do before creating a model
df_data = df_data.withColumnRenamed("Est Income", "EstIncome")

In [6]:
# Print out the top 20 records
df_data.show()

+---+------+------+---------+---+-------------+--------------------+------+-----+
| ID|Gender|Status|EstIncome|Age|LocalBilltype|LongDistanceBilltype| Usage|CHURN|
+---+------+------+---------+---+-------------+--------------------+------+-----+
|  1|     F|     S|  38000.0| 24|       Budget|      Intnl_discount|229.64|    T|
|  6|     M|     M|  29616.0| 49|    FreeLocal|            Standard| 75.29|    T|
|  8|     M|     M|  19732.8| 51|    FreeLocal|            Standard| 47.25|    T|
| 11|     M|     S|    96.33| 56|       Budget|            Standard| 59.01|    T|
| 14|     F|     M|  52004.8| 25|       Budget|      Intnl_discount| 28.14|    T|
| 17|     M|     M|  53010.8| 19|    FreeLocal|            Standard| 58.87|    T|
| 18|     M|     M|  75004.5| 65|       Budget|      Intnl_discount| 58.72|    T|
| 21|     M|     M|  19749.3| 60|       Budget|            Standard| 34.17|    T|
| 22|     M|     S|  57626.9| 44|       Budget|            Standard| 48.35|    T|
| 23|     M|    

As you can see, the space in the 'Est Income' has been removed. The new column name is 'EstIncome'. 

In [8]:
# Count all records
print("Number of records: " + str(df_data.count()))

Number of records: 2066


As per expected, the data set contains 2066 records.

The next step provides a quick way for data scientist to review and evaluate the quality of data before modeling. 
Data preparation and data understanding are the most time-consuming tasks in the data mining process.  

In [9]:
# Statistics summary 
df_data.toPandas().describe()

,ID,EstIncome,Age,Usage
count,2066.000000,2066.000000,2066.000000,2066.000000
mean,1902.820910,51514.070465,42.802033,75.907696
std,1094.820556,30805.652721,14.919932,59.787475
min,1.000000,96.330000,12.000000,0.680000
25%,973.250000,21021.600000,30.250000,34.190000
50%,1880.500000,55860.000000,46.000000,57.110000
75%,2833.750000,78000.000000,54.000000,107.020000
max,3825.000000,120000.000000,77.000000,361.880000


<a id="model"></a>
## 3. Create an Apache Spark machine learning model

In this section you will learn how to:

- [3.1 Prepare data](#prep)
- [3.2 Create an Apache Spark machine learning pipeline](#pipe)
- [3.3 Train a model](#train)


**For more information on Apache Spark machine learning check out the following links:**
- https://spark.apache.org/docs/2.3.0/ml-guide.html
- https://spark.apache.org/docs/2.3.0/ml-pipeline.html

### 3.1 Prepare data<a id="prep"></a>

In this subsection you will split your data into: 
- train data set
- test data set
- predict data set

In [10]:
# Split data into train(70%), test(20%) and predict(10%) datasets
train_data, test_data, predict_data = df_data.randomSplit([0.7,0.2,0.1], seed=6)
train_data.cache()
test_data.cache()
predict_data.cache()

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))
print("Number of prediction records : " + str(predict_data.count()))

Number of training records: 1431
Number of testing records : 407
Number of prediction records : 228


As you can see your data has been successfully split into three data sets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation and is used to test the assumptions of the model.
-  The predict data set will be used for prediction.

### 3.2 Create the pipeline<a id="pipe"></a>

In this section you will create an Apache Spark machine learning pipeline and then train the model.

In the first step, you need to import the Apache Spark machine learning packages that will be needed in the subsequent steps.

In [11]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In the following step, use the StringIndexer transformer to convert all the string fields to numeric ones.

In [12]:
# Prepare string variables so that they can be used by the decision tree algorithm
# StringIndexer encodes a string column of labels to a column of label indices
SI1 = StringIndexer(inputCol='Gender', outputCol='GenderEncoded')
SI2 = StringIndexer(inputCol='Status',outputCol='StatusEncoded')
SI3 = StringIndexer(inputCol='LocalBilltype',outputCol='LocalBilltypeEncoded')
SI4 = StringIndexer(inputCol='LongDistanceBilltype',outputCol='LongDistanceBilltypeEncoded')
labelIndexer = StringIndexer(inputCol='CHURN', outputCol='label').fit(df_data)

# Pipelines API requires that input variables are passed in a vector
# In the following step, create a feature vector by combining all features together.
assembler = VectorAssembler(inputCols=["GenderEncoded", "StatusEncoded", "LocalBilltypeEncoded", \
                                       "LongDistanceBilltypeEncoded", "EstIncome", "Age", "Usage"], \
                            outputCol="features")

Next, define estimators you want to use for classification. Random Forest is used in the following example.

In [13]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

Finally, convert the indexed labels back to original labels.

In [14]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=labelIndexer.labels)

Now build the pipeline. A pipeline consists of transformers and an estimator.

In [15]:
pipeline_rf = Pipeline(stages=[SI1,SI2,SI3,SI4,labelIndexer, assembler, rf, labelConverter])

### 3.3 Train the model<a id="train"></a>

Now, you can train your Random Forest model by using the previously defined **pipeline** and **train data**.

In [16]:
train_data.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- EstIncome: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- LocalBilltype: string (nullable = true)
 |-- LongDistanceBilltype: string (nullable = true)
 |-- Usage: double (nullable = true)
 |-- CHURN: string (nullable = true)



In [17]:
model_rf = pipeline_rf.fit(train_data)

In [18]:
model_rf.transform(test_data)

DataFrame[ID: int, Gender: string, Status: string, EstIncome: double, Age: int, LocalBilltype: string, LongDistanceBilltype: string, Usage: double, CHURN: string, GenderEncoded: double, StatusEncoded: double, LocalBilltypeEncoded: double, LongDistanceBilltypeEncoded: double, label: double, features: vector, rawPrediction: vector, probability: vector, prediction: double, predictedLabel: string]

You can check your **model accuracy** now. Use **test data** to evaluate the model.

In [19]:
results_rf = model_rf.transform(test_data)
results_rf =results_rf.select(results_rf["ID"],results_rf["CHURN"],results_rf["label"],results_rf["predictedLabel"],results_rf["prediction"],results_rf["probability"])
results_rf.toPandas().head(6)

,ID,CHURN,label,predictedLabel,prediction,probability
0,6,T,1.0,F,0.0,"[0.5785767623969055, 0.4214232376030946]"
1,21,T,1.0,F,0.0,"[0.6769011031529951, 0.323098896847005]"
2,22,T,1.0,F,0.0,"[0.6719045383276748, 0.3280954616723252]"
3,29,T,1.0,F,0.0,"[0.695340897847493, 0.304659102152507]"
4,79,T,1.0,F,0.0,"[0.5190534205401753, 0.4809465794598246]"
5,84,T,1.0,T,1.0,"[0.4191694062245329, 0.5808305937754671]"


In [20]:
# Model Evaluation
'Precision model = {:.2f}.'.format(results_rf.filter(results_rf.label == results_rf.prediction).count() / float(results_rf.count()))

'Precision model = 0.61.'

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label", metricName="areaUnderROC")
'Area under ROC curve = {:.2f}.'.format(evaluator.evaluate(results_rf))

'Area under ROC curve = 0.51.'

You can tune your model now to achieve better accuracy. For simplicity, the tuning example is omitted in this example.

<a id="persistence"></a>
## 4. Store the model in the WML repository

In this section you will learn how to use Python client libraries to store your pipeline and model in WML repository.

- [4.1 Import the libraries](#lib)
- [4.2 Save pipeline and model](#save)
- [4.3 Load the model](#load)

### 4.1 Import the libraries<a id="lib"></a>

First, you must install and import the `watson-machine-learning-client` libraries.

**Note**: Python 3.5 and Apache Spark 2.3 is required.

In [22]:
!rm -rf $PIP_BUILD/watson-machine-learning-client

In [23]:
!pip install watson-machine-learning-client --upgrade

  Using cached https://files.pythonhosted.org/packages/b0/53/6bdafebda644e2245fc860ea99300104523665203fc18f1af18bf5124e31/watson_machine_learning_client-1.0.328-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/65/47/7e02164a2a3db50ed6d8a6ab1d6d60b69c4c3fdf57a284257925dfc12bda/requests-2.19.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/24/11/8968863167de09e464672010b59b8e8848bb142c49538711cd3e24c07c59/lomond-0.3.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/df/f7/04fee6ac349e915b82171f8e23cee63644d83663b34c539f7a09aed18f9e/certifi-2018.8.24-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5d/d4/6e9c56a561f1d27407bf29318ca43f36ccaa289271b805a30034eb3a8ec4/pandas-0.23.4-cp35-cp35m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/79/43/19c9fee28110cd47f73e6bc596394337fe9f3e5825b4de402bbf30b3beb5/tqdm-4.26.0-py2.py3-none-any.whl
  Using cached https:/

In [24]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

Authenticate to the Watson Machine Learning service on IBM Cloud.

**Tip**: Authentication information (your credentials) can be found in the <a href="https://console.bluemix.net/docs/services/service_credentials.html#service_credentials" target="_blank" rel="noopener no referrer">Service credentials</a> tab of the service instance that you created on IBM Cloud. 

If you cannot see the **instance_id** field in **Service Credentials**, click **New credential (+)** to generate new authentication information. 

**Action**: Enter your Watson Machine Learning service instance credentials here.

In [25]:
wml_credentials={
  "url": "https://ibm-watson-ml.mybluemix.net",
  "username": "ad02bdee-1526-4813-9180-f5906e021656",
  "password": "c46fdf43-1ae7-4b44-8483-b3b4702183e0",
  "instance_id": "45c8ab27-1720-46a0-8ec7-75d59af433e4"
}

In [26]:
client = WatsonMachineLearningAPIClient(wml_credentials)

### 4.2 Save the pipeline and model<a id="save"></a>

In this subsection you will learn how to save pipeline and model artifacts to your Watson Machine Learning instance.

In [27]:
published_model_details = client.repository.store_model(model=model_rf, meta_props={client.repository.ModelMetaNames.NAME:'Customer Churn Prediction model'}, training_data=train_data, pipeline=pipeline_rf)

In [28]:
model_uid = client.repository.get_model_uid(published_model_details)
print(model_uid)

2ab5d69f-9689-4358-8d4e-8adfefc3e133


Get saved model metadata from Watson Machine Learning.

**Tip**: Use `client.repository.ModelMetaNames.show()` to get the list of available props.

In [29]:
client.repository.ModelMetaNames.show()

-----------------------  ----  --------
META_PROP NAME           TYPE  REQUIRED
NAME                     str   Y
DESCRIPTION              str   N
AUTHOR_NAME              str   N
FRAMEWORK_NAME           str   N
FRAMEWORK_VERSION        str   N
FRAMEWORK_LIBRARIES      list  N
RUNTIME_NAME             str   N
RUNTIME_VERSION          str   N
TRAINING_DATA_REFERENCE  dict  N
EVALUATION_METHOD        str   N
EVALUATION_METRICS       list  N
OUTPUT_DATA_SCHEMA       dict  N
LABEL_FIELD              str   N
TRANSFORMED_LABEL_FIELD  str   N
RUNTIME_UID              str   N
TRAINING_DEFINITION_URL  str   N
-----------------------  ----  --------


In [30]:
# You can get the metadata of stored model 
print(client.repository.get_model_details(model_uid))

{'entity': {'latest_version': {'url': 'https://us-south.ml.cloud.ibm.com/v3/ml_assets/models/2ab5d69f-9689-4358-8d4e-8adfefc3e133/versions/05b0e51e-326d-41d7-aaa5-eacc9c58edfa', 'guid': '05b0e51e-326d-41d7-aaa5-eacc9c58edfa', 'created_at': '2018-09-21T03:46:26.178Z'}, 'learning_iterations_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/45c8ab27-1720-46a0-8ec7-75d59af433e4/published_models/2ab5d69f-9689-4358-8d4e-8adfefc3e133/learning_iterations', 'output_data_schema': {'fields': [{'name': 'ID', 'type': 'integer', 'nullable': True, 'metadata': {}}, {'name': 'Gender', 'type': 'string', 'nullable': True, 'metadata': {}}, {'name': 'Status', 'type': 'string', 'nullable': True, 'metadata': {}}, {'name': 'EstIncome', 'type': 'double', 'nullable': True, 'metadata': {}}, {'name': 'Age', 'type': 'integer', 'nullable': True, 'metadata': {}}, {'name': 'LocalBilltype', 'type': 'string', 'nullable': True, 'metadata': {}}, {'name': 'LongDistanceBilltype', 'type': 'string', 'nullable': True,

### 4.3 Load the model<a id="load"></a>

In this subsection you will learn how to load a saved model from a specified instance of Watson Machine Learning.

In [31]:
loaded_model = client.repository.load(model_uid)

<a id="prediction"></a>
## 5. Predict locally 

In this section you will learn how to score test data using the loaded model and visualize the prediction results with the plotly package.

- [5.1 Make a local prediction using previously loaded model and test data](#local)

### 5.1 Make a local prediction using previously loaded model and test data<a id="local"></a>

In this subsection you will score the *predict_data* data set.

In [32]:
predictions = loaded_model.transform(predict_data)

Preview the results by calling the *show()* method on the predictions DataFrame.

In [34]:
predictions.show(5)

+---+------+------+---------+---+-------------+--------------------+------+-----+-------------+-------------+--------------------+---------------------------+-----+--------------------+--------------------+--------------------+----------+--------------+
| ID|Gender|Status|EstIncome|Age|LocalBilltype|LongDistanceBilltype| Usage|CHURN|GenderEncoded|StatusEncoded|LocalBilltypeEncoded|LongDistanceBilltypeEncoded|label|            features|       rawPrediction|         probability|prediction|predictedLabel|
+---+------+------+---------+---+-------------+--------------------+------+-----+-------------+-------------+--------------------+---------------------------+-----+--------------------+--------------------+--------------------+----------+--------------+
| 14|     F|     M|  52004.8| 25|       Budget|      Intnl_discount| 28.14|    T|          0.0|          0.0|                 0.0|                        1.0|  1.0|[0.0,0.0,0.0,1.0,...|[10.9841382434556...|[0.54920691217278...|       0.0|

In [35]:
# Converting to Pandas data
predictions.toPandas().head(6)

,ID,Gender,Status,EstIncome,Age,LocalBilltype,LongDistanceBilltype,Usage,CHURN,GenderEncoded,StatusEncoded,LocalBilltypeEncoded,LongDistanceBilltypeEncoded,label,features,rawPrediction,probability,prediction,predictedLabel
0,14,F,M,52004.8,25,Budget,Intnl_discount,28.14,T,0.0,0.0,0.0,1.0,1.0,"[0.0, 0.0, 0.0, 1.0, 52004.8, 25.0, 28.14]","[10.984138243455671, 9.01586175654433]","[0.5492069121727836, 0.4507930878272165]",0.0,F
1,18,M,M,75004.5,65,Budget,Intnl_discount,58.72,T,1.0,0.0,0.0,1.0,1.0,"[1.0, 0.0, 0.0, 1.0, 75004.5, 65.0, 58.72]","[13.577199813352381, 6.422800186647619]","[0.678859990667619, 0.32114000933238096]",0.0,F
2,40,F,S,28589.1,16,FreeLocal,Standard,100.28,T,0.0,1.0,1.0,0.0,1.0,"[0.0, 1.0, 1.0, 0.0, 28589.1, 16.0, 100.28]","[13.005515129481282, 6.99448487051872]","[0.6502757564740641, 0.349724243525936]",0.0,F
3,52,F,M,67388.0,53,Budget,Standard,96.33,T,0.0,0.0,0.0,0.0,1.0,"(0.0, 0.0, 0.0, 0.0, 67388.0, 53.0, 96.33)","[10.110702359972008, 9.889297640027992]","[0.5055351179986004, 0.4944648820013996]",0.0,F
4,74,M,M,39229.1,26,Budget,Intnl_discount,47.10,T,1.0,0.0,0.0,1.0,1.0,"[1.0, 0.0, 0.0, 1.0, 39229.1, 26.0, 47.1]","[12.032805295847611, 7.967194704152391]","[0.6016402647923805, 0.3983597352076195]",0.0,F
5,89,F,S,38280.0,34,FreeLocal,Intnl_discount,86.61,T,0.0,1.0,1.0,1.0,1.0,"[0.0, 1.0, 1.0, 1.0, 38280.0, 34.0, 86.61]","[9.41976358934997, 10.58023641065003]","[0.4709881794674985, 0.5290118205325015]",1.0,T


By tabulating a count, you can see which product line is the most popular.

In [36]:
predictions.select("predictedLabel").groupBy("predictedLabel").count().show(truncate=False)

+--------------+-----+
|predictedLabel|count|
+--------------+-----+
|F             |185  |
|T             |43   |
+--------------+-----+



<a id="scoring"></a>
## 6. Deploy and score in a Cloud

In this section you will learn how to create online scoring and to score a new data record using the `watson-machine-learning-client`.

**Note:** You can also use the REST API to deploy and score.
For more information about REST APIs, see the [Swagger Documentation](http://watson-ml-api.mybluemix.net/).

#### Create online deployment for published model.

In [37]:
deployment_details = client.deployments.create(model_uid, name='Customer Churn Risk Model Deployment')



#######################################################################################

Synchronous deployment creation for uid: '2ab5d69f-9689-4358-8d4e-8adfefc3e133' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='6a6319ad-6de2-42f2-b082-1dc82de9309c'
------------------------------------------------------------------------------------------------




#### Create an online scoring endpoint. 

In [38]:
scoring_url = client.deployments.get_scoring_url(deployment_details)
print(scoring_url)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/45c8ab27-1720-46a0-8ec7-75d59af433e4/deployments/6a6319ad-6de2-42f2-b082-1dc82de9309c/online


Now, you can send new scoring records (new data) for which you would like to get predictions. To do that, run the following sample code: 

In [39]:
payload_scoring = {"fields": ["GENDER","STATUS","EstIncome","Age", "LOCALBILLTYPE", "LONGDISTANCEBILLTYPE", "Usage", \
                              "GenderEncoded", "StatusEncoded", "LocalBilltypeEncoded", "LongDistanceBilltypeEncoded"], \
                   "values": \
                             [["M","S", 25000.33, 23,"Budget", "Standard", 100, 1.0, 1.0, 0.0, 0.0]]}

client.deployments.score(scoring_url, payload_scoring)

{'fields': ['GENDER',
  'STATUS',
  'EstIncome',
  'Age',
  'LOCALBILLTYPE',
  'LONGDISTANCEBILLTYPE',
  'Usage',
  'GenderEncoded',
  'StatusEncoded',
  'LocalBilltypeEncoded',
  'LongDistanceBilltypeEncoded',
  'features',
  'rawPrediction',
  'probability',
  'prediction',
  'predictedLabel'],
 'values': [['M',
   'S',
   25000.33,
   23,
   'Budget',
   'Standard',
   100.0,
   1.0,
   1.0,
   0.0,
   0.0,
   [1.0, 1.0, 0.0, 0.0, 25000.33, 23.0, 100.0],
   [14.901902807271412, 5.098097192728589],
   [0.7450951403635706, 0.25490485963642945],
   0.0,
   'F']]}

As you can see we predict that the 23 year old male who is single with estmated income AUD25000.33 who is currently on \'Budget\' local bill type and \'Standard\' long distance bill type with AUD100 usage is about 66\% not likely to churn. 

<a id="summary"></a>
## 7. Summary and next steps     

You successfully completed this notebook! 
 
You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. 
 
Check out our [Online Documentation](https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html) for more samples, tutorials, documentation, how-tos, and blog posts. 

Copyright © 2017, 2018 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>